In [4]:
from PIL import Image
from skimage import io

import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.externals import joblib
import numpy as np

In [5]:
class PathFinder: 
    def __init__(self,model_source,video_source=0):
        self.clf = joblib.load(model_source)
        self.cap=cv2.VideoCapture(video_source)
            
    def __hog(self,img,step):
        from cv2 import HOGDescriptor
        winSize = (16,16)
        blockSize = (16,16)
        blockStride = (8,8)
        cellSize = (8,8)
        nbins = 9
        derivAperture = 1
        winSigma = 4.
        histogramNormType = 0
        L2HysThreshold = 2.0000000000000001e-01
        gammaCorrection = 0
        nlevels = 64
        hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                            histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
        winStride = (8,8)
        padding = (16,16)
        hist = hog.compute(img,winStride,padding)
        return np.reshape(hist, 4356)
    
    def StartRide(self):
        frame=self.cap.read()[1]
        while(self.cap.isOpened()):
            frame=self.cap.read()[1]
            frame = cv2.GaussianBlur(frame,(3,3),0)
            predict=self.__detectSigns(frame)
            #HERE ADD YOUR MODULES
            if predict != []:
                if predict[0] == 0:
                    print('forward')
                if predict[0] == 1:
                    print('left')
                if predict[0] == 2:
                    print('leftForward')
                if predict[0] == 3:
                    print('stop')
                if predict[0] == 4:
                    print('right')
                if predict[0] == 5:
                    print('rightForward')
            cv2.imshow('frame2',frame)
            if cv2.waitKey(2) & 0xFF == ord("q"):
                break     
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __detectSigns(self,frame):
        frame = cv2.resize(frame,(320, 240), interpolation = cv2.INTER_CUBIC)
        r,g,b = cv2.split(frame)    
        circles1 = cv2.HoughCircles(b,cv2.HOUGH_GRADIENT,1,50,param1=100,param2=75,minRadius=20,maxRadius=100)
        circles2 = cv2.HoughCircles(r,cv2.HOUGH_GRADIENT,1,50,param1=100,param2=85,minRadius=20,maxRadius=100)
        pred=[]
        if circles1 is not None:q
            circles1 = np.uint16(np.around(circles1))
            for i in circles1[0,:]:
                cut_frame = frame[(i[1]-i[2]-5):(i[1]+i[2]+5),(i[0]-i[2]-5):(i[0]+i[2]+5)]
                cut_frame = cv2.resize(cut_frame,(64, 64), interpolation = cv2.INTER_CUBIC)
                hog=self.__hog(cut_frame,8)
                k = self.clf.predict_proba([hog])
                for j in k[0]:
                    if j > 0.8:
                        l = np.array([hog])
                        h = self.clf.predict(l)
                        pred.append(h[0])   
        if circles2 is not None:
            circles2 = np.uint16(np.around(circles2))
            for i in circles2[0,:]:
                cut_frame = frame[(i[1]-i[2]-5):(i[1]+i[2]+5),(i[0]-i[2]-5):(i[0]+i[2]+5)]
                cut_frame = cv2.resize(cut_frame,(64, 64), interpolation = cv2.INTER_CUBIC)
                hog=self.__hog(cut_frame,8)
                k = self.clf.predict_proba([hog])
                for j in k[0]:
                    if j > 0.8:
                        l = np.array([hog])
                        h = self.clf.predict(l)
                        pred.append(h[0])
        return pred 

In [6]:
asdf = PathFinder('C:\\Users\\pakli\\Downloads\\RoadSigns\\filename.pkl')
asdf.StartRide()